# Test skill

In [3]:
from characters.ken import Ken
import time

# Choose attack mode: 'modern' or 'classic'
ken = Ken(attack_mode='modern')

# Test Ken's basic attacks
def test_ken_basic():
    ken.light()  # Perform light attack
    time.sleep(1)
    ken.medium()  # Perform medium attack
    time.sleep(1)
    ken.heavy()  # Perform heavy attack
    time.sleep(1)

# Test Ken's special moves
def test_ken_special_moves():
    ken.hadouken()  # Perform Hadouken
    time.sleep(1)
    ken.shoryuken()  # Perform Shoryuken
    time.sleep(1)
    ken.dragonlash_kick()  # Perform Dragonlash Kick
    time.sleep(1)

# Test Ken's defense techniques
def test_ken_defense():
    if ken.attack_mode == 'modern':
        ken.impl.drive_parry()  # Use Drive Parry in modern mode
    else:
        ken.impl.parry()  # Use regular Parry in classic mode
    time.sleep(1)

# Test Ken's power burst technique
def test_ken_burst():
    if ken.attack_mode == 'modern':
        ken.impl.drive_impact()  # Use Drive Impact in modern mode
    else:
        ken.impl.burst()  # Use Burst in classic mode
    time.sleep(1)

# Test Ken's movement
def test_ken_movement():
    # print("Tap move: Jump, Left, Crouch, Right")
    # ken.impl.move_jump()
    # time.sleep(1)
    # ken.impl.move_left()
    # time.sleep(1)
    # ken.impl.move_crouch()
    # time.sleep(1)
    # ken.impl.move_right()
    # time.sleep(1)

    print("Hold move: Hold Left -> Hold Right")
    ken.impl.hold_left()
    time.sleep(2)  # Hold for 2 seconds
    ken.impl.release_left()

    time.sleep(1)

    ken.impl.hold_right()
    time.sleep(2)
    ken.impl.release_right()
    
    time.sleep(1)

    ken.impl.hold_crouch()
    time.sleep(2)
    ken.impl.release_crouch()
    
    time.sleep(1)

    ken.impl.move_jump()


if __name__ == "__main__":
    time.sleep(1)
    ken = Ken(attack_mode='modern')
    for n in range(10):
        test_ken_basic()
        time.sleep(1)

## TEST output player HP

In [ ]:
import cv2
import keyboard
from image_processing.screen_capture import screen_capture
from image_processing.player_hp_detector import L_player_HP, R_player_HP

while True:
    player_blood = screen_capture((115, 85, 1165, 100))
    cv2.imshow("123", player_blood)
    print(f"\rLeft player HP: {L_player_HP(player_blood)}, Right player HP: {R_player_HP(player_blood)}", end='')
    cv2.waitKey(1)
    if keyboard.is_pressed('q'):
        break
cv2.destroyAllWindows()